In [ ]:
import db

from ipyleaflet import (
    Map,
    Marker,
    Popup,
    LayerGroup,
    WidgetControl,
    AwesomeIcon,
    AntPath,
)
from ipywidgets import HTML, IntSlider, Button, Layout

In [ ]:
top_trips_query = """
    with rides as (
        select start_station_id, end_station_id, count(*) as num_rides
        
        from divvy_station_trips e 
        
        group by 1,2
    )
    
    select r.*
    
    , start_s.name as start_name
    , start_s.lat as start_lat
    , start_s.lon as start_lon
    
    
    , end_s.name as end_name
    , end_s.lat as end_lat
    , end_s.lon as end_lon
    
    from rides r
    
    left join divvy_stations start_s on start_s.station_id = r.start_station_id
    left join divvy_stations end_s on end_s.station_id = r.end_station_id
    
    order by num_rides desc
"""
db.query(top_trips_query)

In [ ]:
m = Map(layout=Layout(height="1000px"))


def reset_zoom(self):
    m.center = (41.8, -87.5)
    m.zoom = 10


reset_zoom(None)

map_markers = LayerGroup()
m.add(map_markers)

button = Button(icon="home", tooltip="Reset Zoom", layout=Layout(width="35px"))
button.on_click(reset_zoom)
m.add(WidgetControl(widget=button, position="topleft"))

slider = IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description="Limit:",
    style=dict(description_width="100px"),
    continuous_update=False,
)
m.add(WidgetControl(widget=slider, position="topright"))


display(m)

In [ ]:
def draw_markers(change):
    map_markers.clear_layers()

    top_trips = db.query(f"{top_trips_query} limit {int(change['new'])}").fetchall()

    for row in top_trips:
        message = HTML()
        message.value = f"{row[3]}"
        map_markers.add(
            Marker(
                location=(row[4], row[5]),
                draggable=False,
                popup=message,
                icon=AwesomeIcon(
                    name="bicycle",
                    marker_color="blue",
                ),
            )
        )

        ant_path = AntPath(
            locations=[[row[4], row[5]], [row[7], row[8]]],
            dash_array=[1, 20],
            delay=3000,
            color="#7590ba",
            pulse_color="#3f6fba",
        )

        map_markers.add(ant_path)

        message = HTML()
        message.value = f"{row[6]}"
        map_markers.add(
            Marker(
                location=(row[7], row[8]),
                draggable=False,
                popup=message,
                icon=AwesomeIcon(
                    name="bicycle",
                    marker_color="blue",
                ),
            )
        )


draw_markers({"new": slider.value})
slider.observe(draw_markers, names="value")